# Extract data from web via API and push it to the database

In [61]:
import pandas as pd
import requests
from pytz import timezone
from datetime import datetime
from keys import MySQL_pass
from keys import OW_API_key  
import sqlalchemy

def retrieve_data():
    connection_string = create_connection_string()
    cities_df = fetch_cities_data(connection_string)
    weather_df = fetch_weather_data(cities_df)
    push_weather_data_to_database(weather_df, connection_string)
    return "Data has been updated"

def create_connection_string():
    schema = "gans_data" # The name of your database
    host = "127.0.0.1"
    user = "root"
    password = MySQL_pass # Your MySQL password
    port = 3306
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def fetch_cities_data(connection_string):
    return pd.read_sql("cities", con=connection_string)

def fetch_weather_data(cities_df):

    berlin_timezone = timezone('Europe/Berlin')
    API_key = OW_API_key #USE YOUR OWN OPEN WHEATHER KEY HERE
    weather_items = []

    for _, city in cities_df.iterrows():

        latitude = city["Latitude"]
        longitude = city["Longitude"]
        city_id = city["City_id"]

        url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric")
        response = requests.get(url)
        weather_data = response.json()

        retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

        for item in weather_data["list"]:
            weather_item = {
              "city_id": city_id,
              "forecast_time": item.get("dt_txt"),
              "outlook": item["weather"][0].get("description", None),
              "temperature": item["main"].get("temp"),
              "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
              "wind_speed": item["wind"].get("speed"),
              "rain_prob": item.get("pop", None),
              "data_retrieved_at": retrieval_time
            }
            weather_items.append(weather_item)

    weather_df = pd.DataFrame(weather_items)
    weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
    weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

    return weather_df

def push_weather_data_to_database(weather_df, connection_string):

    weather_df.to_sql('weather',
                    if_exists='append',
                    con=connection_string,
                    index=False)

ImportError: cannot import name 'OW_API_key' from 'keys' (/Users/janinamarc/Desktop/Janina/WBS/Data Acquistition/Musterlösung Lokal/keys.py)

In [63]:
from keys import OW_API_key
print(OW_API_key)  # Testau

ImportError: cannot import name 'OW_API_key' from 'keys' (/Users/janinamarc/Desktop/Janina/WBS/Data Acquistition/Musterlösung Lokal/keys.py)

In [26]:
retrieve_data()

NameError: name 'OW_API_key' is not defined

### SUCCESS!